In [1]:
import sys
sys.path.append('..')

import time
import math
import subprocess
import os
import ovr
ovr.initialize(None)
session, luid = ovr.create()

In [37]:
def quaternion_to_axis_angle(q):
    w, x, y, z = q
    angle = 2*math.acos(w)
    ax = angle * x / math.sqrt(1-w*w)
    ay = angle * y / math.sqrt(1-w*w)
    az = angle * z / math.sqrt(1-w*w)
    return ax, ay, az
def axis_angle_to_quaternion(v):
    ax, ay, az = v
    angle = math.sqrt(ax*ax + ay*ay + az*az)
    return math.cos(angle/2), ax*math.sin(angle/2)/angle, ay*math.sin(angle/2)/angle, az*math.sin(angle/2)/angle

In [38]:
def quat_to_neg_quat(q):
    return q[0], -q[1], -q[2], -q[3]

In [39]:
def quaternion_multiply(quaternion1, quaternion0):
    # Q1*Q0
    w0, x0, y0, z0 = quaternion0
    w1, x1, y1, z1 = quaternion1
    return [-x1 * x0 - y1 * y0 - z1 * z0 + w1 * w0,
                x1 * w0 + y1 * z0 - z1 * y0 + w1 * x0,
                -x1 * z0 + y1 * w0 + z1 * x0 + w1 * y0,
                x1 * y0 - y1 * x0 + z1 * w0 + w1 * z0]

In [40]:
def rotate_vector(quaternion, vector):
    a, b, c = vector
    half = quaternion_multiply(quaternion, [0, a, b, c])
    return quaternion_multiply(half,quat_to_neg_quat(quaternion))

In [6]:
def quaternion_delta(quaternion1,quaternion0):
   
    return quaternion_multiply(quaternion1,quat_to_neg_quat(quaternion0))

In [47]:
# Inputs (Controller Base Orientation + Robot Base Orientation + Cordinate rotation from controller to robot)

# Input Controller Base Orientation
ts  = ovr.getTrackingState(session, ovr.getTimeInSeconds(), True)

# Input Robot Base Orientation
rob_base_ori_axis_angle = [math.pi/2,0,0]

# Input Cordinate rotation from controller to robot
cor_rot_in_axis_angle = [math.pi/2,0,0]

# Work on Inputs
Rtouch = ts.HandPoses[1]
b = Rtouch.ThePose.Orientation
con_base_ori = (b[3],b[0],b[1],b[2])
print(Rtouch.AngularVelocity)
rob_base_ori = axis_angle_to_quaternion(rob_base_ori_axis_angle)
cor_rot = axis_angle_to_quaternion(cor_rot_in_axis_angle)

ovr.Vector3f(-0.00542453443632, 0.0243351701647, 0.00657716766)


In [ ]:
#The stream starts here

In [53]:
# Controller Current Orientation in Controller cordinates

ts  = ovr.getTrackingState(session, ovr.getTimeInSeconds(), True)
Rtouch = ts.HandPoses[1]
b = Rtouch.ThePose.Orientation
con_ori = (b[3],b[0],b[1],b[2])

In [54]:
# Delta in Controller Cordinates(rotation between current and base orientation of the controller)

con_delta_rot = quaternion_delta(con_ori,con_base_ori)
con_delta_rot_axis = quaternion_to_axis_angle(con_delta_rot)

(1.0042858361253253, -0.020357023247629388, 0.4187284228334046)


In [ ]:
# Getting Delta in Robot Cordinate

delta_in_rob_cor_axis_angle = rotate_vector(con_delta_rot_axis,cor_rot)
delta_in_rob_cor = axis_angle_to_quaternion(delta_in_rob_cor_axis_angle)

In [ ]:
# Orientation being sent to the Robot
Ori = quaternion_multiply(delta_in_rob_cor,rob_base_ori)

In [ ]:
#stream ends here

In [55]:
sys.stdout.flush()
ovr.destroy(session)
ovr.shutdown()